<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sodapy

In [0]:
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline


In [4]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,00afed66eb1faf25cf4cba07c4431362bff415c6,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,316,0.85647,17031320100,17031839100,32,32,5,0,2.55,7.55,False,1,41.8849871918,-87.6209929134,"{'type': 'Point', 'coordinates': [-87.62099291...",41.8809944707,-87.6327464887,"{'type': 'Point', 'coordinates': [-87.63274648..."
1,08761e3785ec95ff87a9e4cbeb21af756525be87,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,343,1.619033,17031832600,17031081000,7,8,5,0,2.55,7.55,False,1,41.9147473049,-87.6540070286,"{'type': 'Point', 'coordinates': [-87.65400702...",41.9002656868,-87.6321092196,"{'type': 'Point', 'coordinates': [-87.63210921..."
2,0e960727ed7dc1495341ad2c9c7d5b2a50afd664,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,202,0.874813,NaN,NaN,24,24,2.5,0,2.55,5.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598..."
3,1a382898a696e6517320c1ed1209e2ae67fd0b8b,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,830,8.803145,NaN,NaN,41,28,12.5,0,2.55,15.05,False,1,41.794090253,-87.592310855,"{'type': 'Point', 'coordinates': [-87.59231085...",41.874005383,-87.6635175498,"{'type': 'Point', 'coordinates': [-87.66351754..."
4,2137bd3ad154d39079fabc08bc9e992c340c285f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,663,4.113138,NaN,NaN,24,6,7.5,0,2.55,10.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",41.9442266014,-87.6559981815,"{'type': 'Point', 'coordinates': [-87.65599818..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,21d4b04ee4c7881cceb0ed662701d4b0c523e94c,2019-09-27T19:45:00.000,2019-09-27T20:00:00.000,181,0.377614,17031842200,17031842200,8,8,2.5,0,2.55,5.05,False,1,41.9049353016,-87.6499072264,"{'type': 'Point', 'coordinates': [-87.64990722...",41.9049353016,-87.6499072264,"{'type': 'Point', 'coordinates': [-87.64990722..."
999996,19b1cdf33a401bc0334df88d09fea3f8d1f14a6a,2019-09-27T19:45:00.000,2019-09-27T20:00:00.000,892,3.013781,17031160900,17031060300,16,6,2.5,0,2.55,5.05,True,5,41.9500776349,-87.722610871,"{'type': 'Point', 'coordinates': [-87.72261087...",41.9506052324,-87.671332488,"{'type': 'Point', 'coordinates': [-87.67133248..."
999997,23637e35a72c978c6630c7c4f9732cc35a33782f,2019-09-27T19:45:00.000,2019-09-27T20:00:00.000,589,0.964461,17031280100,17031081700,28,8,5,0,2.55,7.55,False,1,41.8853000224,-87.6428084655,"{'type': 'Point', 'coordinates': [-87.64280846...",41.8920421365,-87.6318639497,"{'type': 'Point', 'coordinates': [-87.63186394..."
999998,18b199ce86ed59902d31f064be2d706aea3cc9f0,2019-09-27T19:45:00.000,2019-09-27T20:15:00.000,1807,11.70026445888,17031281900,17031650200,28,65,25,5,2.55,32.55,False,1,41.8792550844,-87.642648998,"{'type': 'Point', 'coordinates': [-87.64264899...",41.7822169695,-87.7301873353,"{'type': 'Point', 'coordinates': [-87.73018733..."


In [5]:
results_df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,00afed66eb1faf25cf4cba07c4431362bff415c6,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,316,0.85647,17031320100,17031839100,32,32,5,0,2.55,7.55,False,1,41.8849871918,-87.6209929134,"{'type': 'Point', 'coordinates': [-87.62099291...",41.8809944707,-87.6327464887,"{'type': 'Point', 'coordinates': [-87.63274648..."
1,08761e3785ec95ff87a9e4cbeb21af756525be87,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,343,1.619033,17031832600,17031081000,7,8,5,0,2.55,7.55,False,1,41.9147473049,-87.6540070286,"{'type': 'Point', 'coordinates': [-87.65400702...",41.9002656868,-87.6321092196,"{'type': 'Point', 'coordinates': [-87.63210921..."
2,0e960727ed7dc1495341ad2c9c7d5b2a50afd664,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,202,0.874813,NaN,NaN,24,24,2.5,0,2.55,5.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598..."
3,1a382898a696e6517320c1ed1209e2ae67fd0b8b,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,830,8.803145,NaN,NaN,41,28,12.5,0,2.55,15.05,False,1,41.794090253,-87.592310855,"{'type': 'Point', 'coordinates': [-87.59231085...",41.874005383,-87.6635175498,"{'type': 'Point', 'coordinates': [-87.66351754..."
4,2137bd3ad154d39079fabc08bc9e992c340c285f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,663,4.113138,NaN,NaN,24,6,7.5,0,2.55,10.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",41.9442266014,-87.6559981815,"{'type': 'Point', 'coordinates': [-87.65599818..."


In [6]:
results_df.columns

Index(['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_census_tract', 'dropoff_census_tract',
       'pickup_community_area', 'dropoff_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total', 'shared_trip_authorized',
       'trips_pooled', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
       'pickup_centroid_location', 'dropoff_centroid_latitude',
       'dropoff_centroid_longitude', 'dropoff_centroid_location'],
      dtype='object')

In [0]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'dropoff_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]

In [13]:
df = results_df.drop(columns=columns_to_drop)
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,00afed66eb1faf25cf4cba07c4431362bff415c6,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,316,0.85647,32,32,5,0,2.55,7.55
1,08761e3785ec95ff87a9e4cbeb21af756525be87,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,343,1.619033,7,8,5,0,2.55,7.55
2,0e960727ed7dc1495341ad2c9c7d5b2a50afd664,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,202,0.874813,24,24,2.5,0,2.55,5.05
3,1a382898a696e6517320c1ed1209e2ae67fd0b8b,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,830,8.803145,41,28,12.5,0,2.55,15.05
4,2137bd3ad154d39079fabc08bc9e992c340c285f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,663,4.113138,24,6,7.5,0,2.55,10.05


In [0]:
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float)  

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
trip_id                   1000000 non-null object
trip_start_timestamp      1000000 non-null datetime64[ns]
trip_end_timestamp        1000000 non-null datetime64[ns]
trip_seconds              1000000 non-null int64
trip_miles                999994 non-null float64
pickup_community_area     935386 non-null object
dropoff_community_area    925485 non-null object
fare                      1000000 non-null float64
tip                       1000000 non-null float64
additional_charges        1000000 non-null float64
trip_total                1000000 non-null float64
dtypes: datetime64[ns](2), float64(5), int64(1), object(3)
memory usage: 83.9+ MB


In [0]:
df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

In [0]:
df.head(25)

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,weekday,start_hour,start_time_block,start_weekday
0,033eb553a267c9672a16307749ba585b2279d950,2019-09-30 23:45:00,2019-09-30 23:45:00,320,1.386507,8,8,5.0,0.0,2.55,7.55,0,23,7,0
1,06d0dc55591d6fafd44943dceb79db36f9e5f10b,2019-09-30 23:45:00,2019-09-30 23:45:00,319,1.403693,58,63,5.0,0.0,2.55,7.55,0,23,7,0
2,30d8197b8eee1730b0af2719dcdb13a64726bd6b,2019-09-30 23:45:00,2019-09-30 23:45:00,335,1.585585,44,44,5.0,0.0,2.55,7.55,0,23,7,0
3,0b4c86af9dfac5bf734ae0a651e8e997bcc00264,2019-09-30 23:45:00,2019-10-01 00:00:00,778,5.879604,7,5,10.0,0.0,2.55,12.55,0,23,7,0
4,322542beb412800793bd4eb7939202f8606246e1,2019-09-30 23:45:00,2019-09-30 23:45:00,342,1.054647,8,32,5.0,0.0,2.55,7.55,0,23,7,0
5,0578c64349ead288a0e93162d2a8ba23d8b37eed,2019-09-30 23:45:00,2019-10-01 00:00:00,919,6.381502,24,21,12.5,0.0,2.55,15.05,0,23,7,0
6,2b069b93577ba278873f97fab8390d13bfa005ba,2019-09-30 23:45:00,2019-10-01 00:00:00,678,6.193247,28,21,10.0,1.0,2.55,13.55,0,23,7,0
7,0a70d458721b657849a32cf95cf55482ba472e6b,2019-09-30 23:45:00,2019-10-01 00:00:00,622,2.672417,8,28,7.5,0.0,2.55,10.05,0,23,7,0
8,31961c6d3cb60f2179d6a700bf7c253aa40d82a6,2019-09-30 23:45:00,2019-10-01 00:00:00,409,1.346678,8,28,5.0,1.0,2.55,8.55,0,23,7,0
9,063e0b787ba079e9626dc0757844d6cc72758c8b,2019-09-30 23:45:00,2019-09-30 23:45:00,633,5.730192,28,35,10.0,0.0,2.55,12.55,0,23,7,0


In [0]:
X_cat = df[['start_weekday', 'start_hour', 'pickup_community_area']]

In [0]:
X_cat = X_cat.fillna('0')

In [41]:
ohe = OneHotEncoder()
ohe.fit(X_cat)
X_cat_ohe = ohe.transform(X_cat)
columns = ohe.get_feature_names(input_features=X_cat.columns)
X_cat_df = pd.DataFrame(X_cat_ohe.todense(), columns=columns)
y = df['fare']

# Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_cat_df, y, test_size=0.25, random_state=12)


# # Combine categorical and continuous features into the final dataframe
# X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled), cat_train_df], axis=1)
# X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled), cat_test_df], axis=1)
X_cat_df

,start_weekday_0,start_weekday_4,start_weekday_5,start_weekday_6,start_hour_0,start_hour_1,start_hour_2,start_hour_3,start_hour_4,start_hour_5,start_hour_6,start_hour_7,start_hour_8,start_hour_9,start_hour_10,start_hour_11,start_hour_12,start_hour_13,start_hour_14,start_hour_15,start_hour_16,start_hour_17,start_hour_18,start_hour_19,start_hour_20,start_hour_21,start_hour_22,start_hour_23,pickup_community_area_0,pickup_community_area_1,pickup_community_area_10,pickup_community_area_11,pickup_community_area_12,pickup_community_area_13,pickup_community_area_14,pickup_community_area_15,pickup_community_area_16,pickup_community_area_17,pickup_community_area_18,pickup_community_area_19,...,pickup_community_area_43,pickup_community_area_44,pickup_community_area_45,pickup_community_area_46,pickup_community_area_47,pickup_community_area_48,pickup_community_area_49,pickup_community_area_5,pickup_community_area_50,pickup_community_area_51,pickup_community_area_52,pickup_community_area_53,pickup_community_area_54,pickup_community_area_55,pickup_community_area_56,pickup_community_area_57,pickup_community_area_58,pickup_community_area_59,pickup_community_area_6,pickup_community_area_60,pickup_community_area_61,pickup_community_area_62,pickup_community_area_63,pickup_community_area_64,pickup_community_area_65,pickup_community_area_66,pickup_community_area_67,pickup_community_area_68,pickup_community_area_69,pickup_community_area_7,pickup_community_area_70,pickup_community_area_71,pickup_community_area_72,pickup_community_area_73,pickup_community_area_74,pickup_community_area_75,pickup_community_area_76,pickup_community_area_77,pickup_community_area_8,pickup_community_area_9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
999996,0.0,1.0,0.0,0.0,

In [42]:
# run multiple regression using statsmodels
X_cat_df = sm.add_constant(X_cat_df)
model = sm.OLS(y,X_cat_df).fit()
model.summary()


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   fare   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     2900.
Date:                Tue, 21 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:17:04   Log-Likelihood:            -3.5649e+06
No. Observations:             1000000   AIC:                         7.130e+06
Df Residuals:                  999896   BIC:                         7.131e+06
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     4.216e+11   9.95e+11      0.424      0.672   -1.53e+12    2.37e+12
start_weekday_0          -3.801e+11   8.97e+11     -0.424      0.672   -2.14e+12    1.38e+12
start_weekday_4          -3.801e+11   8.97e+11     -0.424      0.672   -2.14e+12    1.38e+12
start_weekday_5          -3.801e+11   8.97e+11     -0.424      0.672   -2.14e+12    1.38e+12
start_weekday_6          -3.801e+11   8.97e+11     -0.424      0.672   -2.14e+12    1.38e+12
start_hour_0             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_1             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_2             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_3             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_4             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_5             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_6             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_7             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_8             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_9             -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_10            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_11            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_12            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_13            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_14            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_15            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_16            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_17            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_18            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_19            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_20            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_21            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_22            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
start_hour_23            -4.928e+10   1.16e+11     -0.424      0.672   -2.77e+11    1.79e+11
pickup_community_area_0   7.793e+09   1.84e+10      0.424      0.672   -2.83e+10    4.38e+10
pickup_communi